## Kaggle M5 competition

In [1]:
import pandas as pd
import numpy as np
import glob
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [2]:
from lightgbm import LGBMRegressor
import joblib
import warnings

warnings.filterwarnings('ignore')


In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [4]:
import sys
lib_path = '../../../frameworks/time_series_signals_sound'
sys.path.append(lib_path)
lib_path_2 = '../../../frameworks/'
sys.path.append(lib_path_2)

In [5]:
from modeling import ts_models

In [6]:
glob.glob('../../../data/kaggle/M5/*')

['../../../data/kaggle/M5/sales_train_evaluation.csv',
 '../../../data/kaggle/M5/sell_prices.csv',
 '../../../data/kaggle/M5/calendar.csv',
 '../../../data/kaggle/M5/sales_train_validation.csv',
 '../../../data/kaggle/M5/sample_submission.csv']

In [15]:
calendar = pd.read_csv('../../../data/kaggle/M5/calendar.csv')
#sales = pd.read_csv('../../../data/kaggle/M5/sales_train_evaluation.csv')
#prices = pd.read_csv('../../../data/kaggle/M5/sell_prices.csv')

In [8]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1969 non-null   object
 1   wm_yr_wk      1969 non-null   int64 
 2   weekday       1969 non-null   object
 3   wday          1969 non-null   int64 
 4   month         1969 non-null   int64 
 5   year          1969 non-null   int64 
 6   d             1969 non-null   object
 7   event_name_1  162 non-null    object
 8   event_type_1  162 non-null    object
 9   event_name_2  5 non-null      object
 10  event_type_2  5 non-null      object
 11  snap_CA       1969 non-null   int64 
 12  snap_TX       1969 non-null   int64 
 13  snap_WI       1969 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 215.5+ KB


In [8]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


In [ ]:
sales.info()

In [12]:
# add future
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)

In [13]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df 

In [16]:
#sales = downcast(sales)
#prices = downcast(prices)
calendar = downcast(calendar)

## Baseline

In [16]:
# %%time

# sales_valid = pd.read_csv('../../../data/kaggle/M5/sales_train_validation.csv')
# sample = pd.read_csv('../../../data/kaggle/M5/sample_submission.csv', index_col=0)
# valid = []
# for idx, row in sales_valid.iterrows():
#     valid.append(ts_models.baseline_forecast(row[6:], 
#                                        ftype='season mean', 
#                                        steps=28, 
#                                        n_seasons=4, 
#                                        time_index=False,
#                                        index=sample.columns).to_frame().T)

# valid = pd.concat(valid).reset_index(drop=True)
# valid['id'] = data_valid['id']
    
    
# evalut = []
# for idx, row in sales.iterrows():
#     evalut.append(ts_models.baseline_forecast(row[6:], 
#                                        ftype='season mean', 
#                                        steps=28, 
#                                        n_seasons=4, 
#                                        time_index=False,
#                                        index=sample.columns).to_frame().T)

# evalut = pd.concat(evalut).reset_index(drop=True)
# evalut['id'] = data['id']
    
# submission =  pd.concat([valid, evalut]).set_index('id')
# submission.to_csv('baselin_sub.csv')

In [17]:
%%time
# melt series and merge all data
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left')

CPU times: user 1min 16s, sys: 43.7 s, total: 2min
Wall time: 2min 1s


In [18]:
del sales, prices, calendar
gc.collect()

0

In [21]:
%%time
df['enc'+'item_id'+'mean'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)

CPU times: user 563 ms, sys: 97.3 ms, total: 660 ms
Wall time: 689 ms


In [25]:
df[df['id'] == 'HOBBIES_1_001_CA_1_evaluation']

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,encitem_idmean
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,0.216553
30490,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0,2011-01-30,11101,...,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,0.216553
60980,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,...,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,0.216553
91470,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,...,2011,NaN,NaN,NaN,NaN,1,1,0,NaN,0.216553
121960,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,...,2011,NaN,NaN,NaN,NaN,1,0,1,NaN,0.216553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59882360,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1965,0,2016-06-15,11620,...,2016,NaN,NaN,NaN,NaN,0,1,1,8.382812,0.216553
59912850,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1966,0,2016-06-16,11620,...,2016,NaN,NaN,NaN,NaN,0,0,0,8.382812,0.216553
59943340,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1967,0,2016-06-17,11620,...,2016,NaN,NaN,NaN,NaN,0,0,0,8.382812,0.216553
59973830,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1968,0,2016-06-18,11621,...,2016,NaN,NaN,NaN,NaN,0,0,0,8.382812,0.216553


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60034810 entries, 0 to 60034809
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            category      
 1   item_id       category      
 2   dept_id       category      
 3   cat_id        category      
 4   store_id      category      
 5   state_id      category      
 6   d             object        
 7   sold          int16         
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       category      
 11  wday          int8          
 12  month         int8          
 13  year          int16         
 14  event_name_1  category      
 15  event_type_1  category      
 16  event_name_2  category      
 17  event_type_2  category      
 18  snap_CA       int8          
 19  snap_TX       int8          
 20  snap_WI       int8          
 21  sell_price    float16       
dtypes: category(11), datetime64[ns](1), float16(1), int16(3), int8(5

In [20]:
# make dict 
d_id = dict(zip(df.id.cat.codes, df.id))
d_store_id = dict(zip(df.store_id.cat.codes, df.store_id))

In [21]:
# event coumns preprocessing
list1=['event_name_1','event_type_1','event_name_2','event_type_2']
for i in list1:
    
    df[i] = df[i].cat.add_categories("nan").fillna("nan")
    df[i]=LabelEncoder().fit_transform(df[i]).astype(np.int8)
    df[i]=df[i].astype('category')

In [22]:
# extract days
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)

In [23]:
cols = df.dtypes.index.tolist()
types = df.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category':
        df[cols[i]] = df[cols[i]].cat.codes

In [24]:
df['date'] = df['date'].apply(lambda x: x.strftime('%d')).astype(np.int8)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60034810 entries, 0 to 60034809
Data columns (total 22 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            int16  
 1   item_id       int16  
 2   dept_id       int8   
 3   cat_id        int8   
 4   store_id      int8   
 5   state_id      int8   
 6   d             int16  
 7   sold          int16  
 8   date          int8   
 9   wm_yr_wk      int16  
 10  weekday       int8   
 11  wday          int8   
 12  month         int8   
 13  year          int16  
 14  event_name_1  int8   
 15  event_type_1  int8   
 16  event_name_2  int8   
 17  event_type_2  int8   
 18  snap_CA       int8   
 19  snap_TX       int8   
 20  snap_WI       int8   
 21  sell_price    float16
dtypes: float16(1), int16(6), int8(15)
memory usage: 2.1 GB


## Add extra features

In [26]:
%%time
#lags sold
lags = [28,35,42,49]
for lag in lags:
    df['sold_lag_'+str(lag)] = df.groupby(['id', 
                                           'item_id', 
                                           'dept_id',
                                           'cat_id', 
                                           'store_id', 
                                           'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)

CPU times: user 21.8 s, sys: 3.41 s, total: 25.2 s
Wall time: 26.6 s


In [27]:
# lags event
lags2 = [1,2]
for lag in lags2:
    df['event1_lag_'+str(lag)] = df.groupby(['id', 
                                             'item_id', 
                                             'dept_id', 
                                             'cat_id', 
                                             'store_id', 
                                             'state_id'],as_index=False)['event_name_1']\
                                    .shift(lag)\
                                    .astype(np.float16)
    
    df['event1_lag_'+str(lag)].fillna(100, inplace=True)
    df['event1_lag_'+str(lag)] = df['event1_lag_'+str(lag)].astype(np.int8)
    df['event1_lag_'+str(lag)] = df['event1_lag_'+str(lag)].astype('category')

In [28]:
%%time
# mean sold of item by all stores
df['item_sold_avg'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)    
#df['state_sold_avg'] = df.groupby('state_id')['sold'].transform('mean').astype(np.float16)
#df['store_sold_avg'] = df.groupby('store_id')['sold'].transform('mean').astype(np.float16)

# mean sold  by category, department, cat + dep, store + item, cat + item, dep+item, store+cat+dep
df['cat_sold_avg'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
df['dept_sold_avg'] = df.groupby('dept_id')['sold'].transform('mean').astype(np.float16)
df['cat_dept_sold_avg'] = df.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
df['store_item_sold_avg'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['cat_item_sold_avg'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['dept_item_sold_avg'] = df.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
#df['state_store_sold_avg'] = df.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
#df['state_store_cat_sold_avg'] = df.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
df['store_cat_dept_sold_avg'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)

CPU times: user 13 s, sys: 2.05 s, total: 15 s
Wall time: 15.1 s


In [29]:
%%time
# replace week number to atarted from zero  
df['wm_yr_wk_linear']=LabelEncoder().fit_transform(df['wm_yr_wk'].values).astype(np.int16)
df.drop(['wm_yr_wk'], axis=1, inplace=True)

CPU times: user 2.84 s, sys: 1.05 s, total: 3.89 s
Wall time: 4.03 s


In [30]:
%%time
# add week price lag and diff
df['price_lag'] = df.groupby(['id', 
                              'item_id', 
                              'dept_id', 
                              'cat_id', 
                              'store_id', 
                              'state_id'],as_index=False)['sell_price'].shift(7).astype(np.float16)
# TODO sell price contain nan price-diff - too
df['price-diff']=df['price_lag']-df['sell_price']
df.drop(['price_lag'], axis=1, inplace=True)

CPU times: user 6.82 s, sys: 1.12 s, total: 7.94 s
Wall time: 7.95 s


In [31]:
%%time
# add value of cents - 99 effect ?
df['sell_price'].fillna(-1,inplace=True)
df['decimal']=df['sell_price'].apply(lambda x: 100*(x-int(x))).astype(np.int16)
df['sell_price'].replace(-1,np.nan,inplace=True)

CPU times: user 17.2 s, sys: 1.14 s, total: 18.3 s
Wall time: 18.4 s


In [32]:
%%time
# calculate expanding mean price with window 2
df['expanding_price_mean'] = df.groupby(['id', 
                                         'item_id', 
                                         'dept_id', 
                                         'cat_id', 
                                         'store_id', 
                                         'state_id'])['sell_price']\
                                .transform(lambda x: x.expanding(2).mean())\
                                .astype(np.float16)
# calculate difference
df['diff_moving_mean']=df['expanding_price_mean']-df['sell_price']

# drop expanding mean
df.drop(['expanding_price_mean'], axis=1, inplace=True)

CPU times: user 39.7 s, sys: 16.2 s, total: 55.8 s
Wall time: 1min 16s


In [33]:
%%time
# optimize mamory
df['price-diff']=df['price-diff'].astype(np.float16)
# drop wdas - same as weekday
df.drop(['wday'], axis=1, inplace=True)
df['decimal']=df['decimal'].astype(np.int8)
# encode year
df['year']=LabelEncoder().fit_transform(df['year']).astype(np.int8)

CPU times: user 2.54 s, sys: 384 ms, total: 2.93 s
Wall time: 3 s


In [34]:
%%time
# ?? sell price  not change - may be erros
df['daily_avg_sold'] = df.groupby(['id', 
                                   'item_id', 
                                   'dept_id', 
                                   'cat_id', 
                                   'store_id', 
                                   'state_id','d'])['sell_price'].transform('mean').astype(np.float16)
# mean price
df['avg_sold'] = df.groupby(['id', 
                             'item_id', 
                             'dept_id', 
                             'cat_id', 
                             'store_id', 
                             'state_id'])['sell_price'].transform('mean').astype(np.float16)
# difference between daily price and avg price (may be mean sells)
df['selling_trend'] = (df['daily_avg_sold'] - df['avg_sold']).astype(np.float16)
df.drop(['daily_avg_sold','avg_sold'],axis=1,inplace=True)

CPU times: user 39.8 s, sys: 22.6 s, total: 1min 2s
Wall time: 1min 3s


In [35]:
%%time
#max price
df['price_max'] = df.groupby(['store_id','item_id'])['sell_price'].transform('max')
#df['price_min'] = df.groupby(['store_id','item_id'])['sell_price'].transform('min')
#df['price_std'] = df.groupby(['store_id','item_id'])['sell_price'].transform('std')
#df['price_mean'] = df.groupby(['store_id','item_id'])['sell_price'].transform('mean')

# relate to max price
df['price_norm'] = df['sell_price']/df['price_max']

# relate price to mean by month and year
#df['price_momentum'] = df['sell_price']/df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
df['price_momentum_m'] = df['sell_price']/df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
df['price_momentum_y'] = df['sell_price']/df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

CPU times: user 10.4 s, sys: 936 ms, total: 11.3 s
Wall time: 11.3 s


In [36]:
#df['rolling_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
#df['expanding_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.expanding(2).mean()).astype(np.float16)
#df['daily_avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','d'])['sold'].transform('mean').astype(np.float16)
#df['avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform('mean').astype(np.float16)
#df['selling_trend'] = (df['daily_avg_sold'] - df['avg_sold']).astype(np.float16)
#df.drop(['daily_avg_sold','avg_sold'],axis=1,inplace=True)

In [37]:
#drop category and state
list3=['cat_id','state_id']
for i in list3:
    df.drop([i], axis=1, inplace=True)

## Preapare data for modeling

In [38]:
%%time
df = df[df['d']>=49]
df.to_pickle('data.pkl')

CPU times: user 11.9 s, sys: 4.06 s, total: 16 s
Wall time: 1min 21s


In [39]:
del df
gc.collect()

0

In [40]:
data = pd.read_pickle('data.pkl')

In [41]:
# split data on train/valid/test sets
valid_csv=data[(data['d']>=1914) & (data['d']<1942)][['id','d','sold']]
test = data[data['d']>=1942][['id','d','sold']]
eval_preds = test['sold']
valid_preds_csv=valid_csv['sold']

In [42]:
cat_column=[]
for i in data.columns:
    if(str(data.dtypes[i])=='category'):
        cat_column.append(i)

In [63]:
%%time
for store in d_store_id:
    # filter data by store
    df = data[data['store_id']==store]
    
    # split train/valid/test sets
    X_train, y_train = df[df['d']<1914].drop('sold',axis=1), df[df['d']<1914]['sold']
    X_valid_csv, y_valid_csv = df[(df['d']>=1914) & (df['d']<1942)].drop('sold',axis=1), df[(df['d']>=1914) & (df['d']<1942)]['sold']
    X_test = df[df['d']>=1942].drop('sold',axis=1)
    
    model = LGBMRegressor(
        learning_rate= 0.05,
        subsample=0.6,
        feature_fraction=0.6,
        num_iterations = 1200,
        max_bin=350,
        num_leaves= 300,
        lambda_l2=0.003,
        max_depth=200,
        min_data_in_leaf= 80,
        force_row_wise= True, #objective='tweedie'
    )
    #fit model
    print('*****Prediction for Store: {}*****'.format(d_store_id[store]))
    model.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_valid_csv,y_valid_csv)],
             eval_metric='rmse',  verbose=100, early_stopping_rounds=20,categorical_feature=cat_column)
    #predict valid values
    valid_preds_csv[X_valid_csv.index] = model.predict(X_valid_csv)
    # predict x_test
    eval_preds[X_test.index] = model.predict(X_test)
    filename = 'model'+str(d_store_id[store])+'.pkl'

    joblib.dump(model, filename)
    del model, X_train, y_train, X_valid_csv, y_valid_csv
    gc.collect()

*****Prediction for Store: CA_1*****
Training until validation scores don't improve for 20 rounds
[100]	training's rmse: 2.42529	training's tweedie: 3.36887	valid_1's rmse: 2.28445	valid_1's tweedie: 4.30605
Early stopping, best iteration is:
[130]	training's rmse: 2.37201	training's tweedie: 3.35605	valid_1's rmse: 2.26755	valid_1's tweedie: 4.30284
*****Prediction for Store: CA_2*****
Training until validation scores don't improve for 20 rounds
[100]	training's rmse: 1.85687	training's tweedie: 2.8356	valid_1's rmse: 2.11245	valid_1's tweedie: 4.39203
Early stopping, best iteration is:
[132]	training's rmse: 1.83072	training's tweedie: 2.82176	valid_1's rmse: 2.08933	valid_1's tweedie: 4.38725
*****Prediction for Store: CA_3*****
Training until validation scores don't improve for 20 rounds
[100]	training's rmse: 3.375	training's tweedie: 3.95914	valid_1's rmse: 2.63403	valid_1's tweedie: 4.85689
Early stopping, best iteration is:
[155]	training's rmse: 3.23963	training's tweedie: 3.9

In [67]:
valid_csv['sold'] = valid_preds_csv
validation = valid_csv[['id','d','sold']]
validation = pd.pivot(validation, index='id', columns='d', values='sold').reset_index()
validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
validation.id = validation.id.map(d_id).str.replace('evaluation','validation')

test['sold'] = eval_preds
evaluation = test[['id','d','sold']]
evaluation = pd.pivot(evaluation, index='id', columns='d', values='sold').reset_index()
evaluation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
evaluation.id = evaluation.id.map(d_id)


submit = pd.concat([validation,evaluation]).reset_index(drop=True)

# increase on 4% ??
for i in range(1,29):
    submit['F'+str(i)] *= 1.04

submit.to_csv('submission_2.csv',index=False)

In [60]:
future = data[data['d']>=1942]

In [61]:
data.isna().sum()

id                                0
item_id                           0
dept_id                           0
store_id                          0
d                                 0
sold                              0
date                              0
weekday                           0
month                             0
year                              0
event_name_1                      0
event_type_1                      0
event_name_2                      0
event_type_2                      0
snap_CA                           0
snap_TX                           0
snap_WI                           0
sell_price                 11439643
sold_lag_28                       0
sold_lag_35                       0
sold_lag_42                       0
sold_lag_49                   30490
event1_lag_1                      0
event1_lag_2                      0
item_sold_avg                     0
cat_sold_avg                      0
dept_sold_avg                     0
cat_dept_sold_avg           

In [62]:
future.isna().sum()

id                         0
item_id                    0
dept_id                    0
store_id                   0
d                          0
sold                       0
date                       0
weekday                    0
month                      0
year                       0
event_name_1               0
event_type_1               0
event_name_2               0
event_type_2               0
snap_CA                    0
snap_TX                    0
snap_WI                    0
sell_price                 0
sold_lag_28                0
sold_lag_35                0
sold_lag_42                0
sold_lag_49                0
event1_lag_1               0
event1_lag_2               0
item_sold_avg              0
cat_sold_avg               0
dept_sold_avg              0
cat_dept_sold_avg          0
store_item_sold_avg        0
cat_item_sold_avg          0
dept_item_sold_avg         0
store_cat_dept_sold_avg    0
wm_yr_wk_linear            0
price-diff                 0
decimal       

In [65]:
train = df[df['d']<1914]

In [66]:
train.isna().sum()

id                               0
item_id                          0
dept_id                          0
store_id                         0
d                                0
sold                             0
date                             0
weekday                          0
month                            0
year                             0
event_name_1                     0
event_type_1                     0
event_name_2                     0
event_type_2                     0
snap_CA                          0
snap_TX                          0
snap_WI                          0
sell_price                 1063518
sold_lag_28                      0
sold_lag_35                      0
sold_lag_42                      0
sold_lag_49                   3049
event1_lag_1                     0
event1_lag_2                     0
item_sold_avg                    0
cat_sold_avg                     0
dept_sold_avg                    0
cat_dept_sold_avg                0
store_item_sold_avg 

In [ ]:
"""MOON PHASE CALCULATION
credits to: https://gist.github.com/miklb/ed145757971096565723
moonphase.py - Calculate Lunar Phase
Author: Sean B. Palmer, inamidst.com
Cf. http://en.wikipedia.org/wiki/Lunar_phase#Lunar_phase_calculation
"""
import math, decimal
dec = decimal.Decimal

def get_moon_phase(d):  # 0=new, 4=full; 4 days/phase
    diff = d - datetime(2001, 1, 1)
    days = dec(diff.days) + (dec(diff.seconds) / dec(86400))
    lunations = dec("0.20439731") + (days * dec("0.03386319269"))
    phase_index = math.floor((lunations % dec(1) * dec(8)) + dec('0.5'))
    return int(phase_index) &amp; 7